In [1]:
import os
import sys
sys.path.append('..') 
import yaml
import pandas as pd
import numpy as np
import pickle
from src.utils.utils_p import YamlParser,Upload_Download_Pickle,Config_Paths,Feature_Selection_Configs,Model_Configs,Create_Labels_Configs
from src.data.CollectData.CollectDataFromCSV import *
from src.preprocessing.Data_Preprocessing import *
from src.combineDatasets.Combine_Datasets import *
from src.featureProcessing.SplitDataset.Split_Train_Test import *
from src.featureProcessing.SplitDataset.Vertical_Split import *
from src.featureProcessing.Feature_Selection.Basic_Elemination import *
from src.featureProcessing.Feature_Selection.Wrapper import *
from src.featureProcessing.Feature_Selection.Hybrid import *
from src.featureProcessing.Feature_Extraction.Normalize import *
from src.featureProcessing.Feature_Extraction.Rolling import *
from src.featureProcessing.Imputation.Basic_Imputation import *
#from src.processing.Model import *
from src.model.Classification.Random_Forest_Tuning import *
from src.model.Classification.XGBoost_Tuning import *
from src.model.Classification.LightGBM_Tuning import *
from src.model.Classification.CatBoost_Tuning import *
from src.model.Classification.SVC_Tuning import *
from src.model.Classification.Voting import *
pd.set_option('display.max_columns',None)
pd.set_option('display.max_rows',None)

In [3]:
path=Config_Paths().get_processed_path()
cpath=Config_Paths().get_combineddatasets_path()
rpath=Config_Paths().get_raw_path()
prepath=Config_Paths().get_intermediate_path()
modelpath=Config_Paths().get_model_path()

In [ ]:
combined_data=pd.read_csv('merged_data_with_all_tags_2020.csv',delimiter='|',header=0)

In [ ]:
#PERIOD yerine YEAR_MONTH_DAY geldi.
LogsT02=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT02')
LabelsT02=Upload_Download_Pickle().download_pickle(prepath, 'Labels_not_normalT02')
LogsT03=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT03')
LogsT52=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT52')


In [ ]:
LogsT02.State_Fault.astype(int).unique()

In [ ]:
LogsT02=pd.merge(LogsT02,LabelsT02[['Pitch systemEv_multiclass','Pitch systemEv','Period']])

In [ ]:
from datetime import datetime, timedelta
LogsT02['onedayPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=1)
LogsT02['fivedaysPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=5)
LogsT02['fiftheendaysPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=15)


In [ ]:
LogsT02=LogsT02.set_index('Period')
tag=LogsT02[LogsT02['Pitch systemEv']=='EVENT_134']


In [ ]:
for i in tag.index:
    print(i)
    print(tag.loc[i,'onedayPeriod'])
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'fiftheendaysPeriod']),'EVENT_134_fifthteen',0)    
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'fivedaysPeriod']),'EVENT_134_five',LogsT02['New_Pitch_systemEv'])  
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'onedayPeriod']),'EVENT_134_one',LogsT02['New_Pitch_systemEv'])
  
    

In [ ]:
LogsT02 ['New_Pitch_systemEv'].unique()

In [ ]:
tag_plot=LogsT02[LogsT02['New_Pitch_systemEv']!=0]

In [ ]:
#CollectData from given dataset
CollectDataFromCSV().upload_all_datasets() 

In [ ]:
col_22=[col.replace(" 22 ", " ") for col in LogsT22.columns]
col_02=[col.replace(" 02 ", " ") for col in LogsT02.columns]

In [ ]:
[items for items in col_02 if items not in col_22]

In [ ]:
[items for items in col_22 if items not in col_02]

In [ ]:
LogsT22=Upload_Download_Pickle().download_pickle(rpath, 'LogsT22')

In [ ]:
FaultsT22=Upload_Download_Pickle().download_pickle(rpath, 'FaultsT22')

In [ ]:
LogsT22.describe()

In [ ]:
# Duration'ı saat'e çevir onu al!
FaultsT22['Duration (hh:mm:ss)'].dtypes

In [ ]:
# Hata ne sıklıkla geliyor bunu al!!
Fault_count=FaultsT22[['Event ID','Category','Description','Lost Production (kWh)']].groupby(['Event ID','Category','Description']).count()
# Hata toplam Lost:
Fault_total_Lost=FaultsT22[['Event ID','Category','Description','Lost Production (kWh)']].groupby(['Event ID','Category','Description']).sum().sort_values(by='Lost Production (kWh)',ascending=False)

## En son hepsini merge et!

In [ ]:
sum_list=LogsT02.isna().sum()

In [ ]:
#CollectData from given dataset
CollectDataFromCSV().upload_all_datasets() 

#Clearn-Normalize-Combine dataset
Data_Preprocessing().preprocessing()

In [ ]:
Features_not_normalT02=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT02')
Labels_not_normalT02=Upload_Download_Pickle().download_pickle(prepath, 'Labels_not_normalT02')
Features_not_normalT22=Upload_Download_Pickle().download_pickle(prepath, 'Features_not_normalT22')
Labels_not_normalT22=Upload_Download_Pickle().download_pickle(prepath, 'Labels_not_normalT22')
Logs_all_fillT22=Upload_Download_Pickle().download_pickle(prepath, 'Logs_all_fillT22')
Logs_all_fillT02=Upload_Download_Pickle().download_pickle(prepath, 'Logs_all_fillT02')

In [ ]:
Labels_not_normalT02["Events_binary"].unique()

In [ ]:
Labels_not_normalT02.columns

In [ ]:
#Combine Multiple Identical Datasets
Combine_Datasets().multi_combine_dataset()

In [ ]:
#PERIOD yerine YEAR_MONTH_DAY geldi.
Features_df=Upload_Download_Pickle().download_pickle(cpath, 'Features_df')
AllLabels_df=Upload_Download_Pickle().download_pickle(cpath, 'AllLabels_df')
Labels_df=Upload_Download_Pickle().download_pickle(cpath, 'Labels_df')


In [ ]:
# Değişkenleri rename ettim:
renamecolumns= {'timestamp':'Period','WTG_ID':'Turbine_Number','Unnamed: 0': 'Period','Power (kW)':'Power_kW','Angle rotor position (°)':'Rotor_angle_degree','Rotor speed (PLC) (rpm)':'Rotor_rpm','BuddyYawOffset (°)': 'BuddyYawOffset_degree','CCU current (A)': 'CCU_current_A','CCU power (kW)': 'CCU_power_kW','CCU reactive power (kVAr)': 'CCU_reactive_power_kVAr','CCU voltage (V)': 'CCU_voltage_V','Circuit breaker cut-ins': 'CCU_Circuit_breaker_cutins','Current A (A)': 'Current_A_A','Current B (A)': 'Current_B_A','Current C (A)': 'Current_C_A','Drip loop power cable temperature 1 (°C)': 'Drip_loop_power_cable_temp_1','Drip loop power cable temperature 2 (°C)': 'Drip_loop_power_cable_temp_2','Drive train acceleration (mm/s²)': 'Drive_train_acc_mm_s2','Generator Temp. 3 (°C)': 'Generator_temp_3_C','Generator speed (CCU) (rpm)': 'CCU_Generator_speed_rpm','Hydraulic prepressure (bar)': 'Hydraulic_prepressure_bar','Line frequency (Hz)': 'Line_freq_Hz','Mainframe acceleration lateral (mm/s²)': 'Mainframe_acc_lateral','Mainframe acceleration vertical (mm/s²)': 'Mainframe_acc_vertical','Mom. D filtered (kNm)': 'Mom_D_filtered','Mom. Q filltered (kNm)': 'Mom_Q_filtered','Moment D direction (kNm)': 'Mom_D_direction','Moment Q direction (kNm)': 'Mom_Q_direction','Nacelle position (°)':'Nacelle_Position','Generator speed (PLC) (rpm)':'Generator_rpm','Torque, actual value (%)':'Torque_act_Percent','Torque, set value (%)':'Torque_set_Percent','Tower acceleration  normal (mm/s²)':'Tower_acc_normal','Tower acceleration lateral (mm/s²)':'Tower_acc_lateral','Temp. tower base (°C)':'Tower_base_temp_C','Temp. Ambient (°C)':'Ambient_temp_C','Windspeed a. 1 (m/s)':'Windspeed_a1_m_s','Temp. gear box bearing (°C)':'gearboxbearing_temp_C','Temp. GBX bearing hollow shaft (°C)':'GBX_bearing_hollow_shaft_temp_C','T_GBX_OIL_1 (°C)':'GBX_OIL1_temp_C','T_GBX_OIL_2 (°C)':'GBX_OIL2_temp_C','T_GBX_T1-HSS (°C)':'GBX_T1_HSS_temp_C','T_GBX_T3-HSS (°C)':'GBX_T3_HSS_temp_C','T_GBX_T1-IMS (°C)':'GBX_T1_IMS_temp_C','T_GBX_T3-IMS (°C)':'GBX_T3_IMS_temp_C','T_GBX_Distr (°C)':'GBX_Distr_temp_C','T_GBX_OIL (°C)':'GBX_OIL_temp_C','Temp. shaft bearing 1 (°C)':'Shaft_bearing1_temp_C','Temp. shaft bearing 2 (°C)':'Shaft_bearing2_temp_C','Prox. sensor 45° (mm)':'Prox_sensor45_mm','Prox. sensor 135° (mm)':'Prox_sensor135_mm','Prox. sensor 225° (mm)':'Prox_sensor225_mm','Prox. sensor 315° (mm)':'Prox_sensor315_mm','Blade 2, actual value A (°)':'Blade2_act_val_A_degree','Blade 2 actual value B (°)':'Blade2_act_val_B_degree','Blade 2, set value (°)':'Blade2_set_val_degree','Blade 3, actual value A (°)':'Blade3_act_val_A_degree','Blade 3 actual value B (°)':'Blade3_act_val_B_degree','Blade 3, set value (°)':'Blade3_set_val_degree','Blade 1, set value (°)':'Blade1_set_val_degree','Blade 1, actual value A (°)':'Blade1_act_val_A_degree','Blade 1 actual value B (°)':'Blade1_act_val_B_degree','Nacelle revolution' :  'Nacelle_rev','Nacelle revolution.1' :  'Nacelle_rev.1','Operating state' :  'Operating_state','Operating state.1' :  'Operating_state_1','Particle counter' :  'Particle_counter','Particle counter.1' :  'Particle_counter_1','Pitch Offset 1 ALC (°)' :  'Pitch_offset_1_ALC_deg','Pitch Offset 1 ALC (°).1' :  'Pitch_offset_1_ALC_deg_1','Pitch Offset 2 ALC (°)' :  'Pitch_offset_2_ALC_deg','Pitch Offset 2 ALC (°).1' :  'Pitch_offset_2_ALC_deg_1','Pitch Offset 3 ALC (°)' :  'Pitch_offset_3_ALC_deg','Pitch Offset 3 ALC (°).1' :  'Pitch_offset_3_ALC_deg_1','Pitch demand baseline (°)' :  'Pitch_demand_baseline_def','Pitch demand baseline (°).1' :  'Pitch_demand_baseline_def_1','PitchOffset Tower feedback (°)' :  'Pitch_offset_tower_feedback','PitchOffset Tower feedback (°).1' :  'Pitch_offset_tower_feedback_1','Power (kW).1' :  'Power_kW_1','Power factor' :  'Power_factor','Power factor, set value' :  'Power_factor_set_value','Reactive power (kVAr)' :  'Reactive_power_kVar','Rotor lock position (°)' :  'Rotor_lock_position','Scope CH 1' :  'Scope_CH_1','Scope CH 2' :  'Scope_CH_2','Scope CH 3' :  'Scope_CH_3','Scope CH 4' :  'Scope_CH_4','State and Fault' :  'State_Fault','Temp. Axis box 1 (°C)' :  'Temp_Axis_box_1','Temp. Axis box 2 (°C)' :  'Temp_Axis_box_2','Temp. Axis box 3 (°C)' :  'Temp_Axis_box_3','Temp. Heat Exchanger CCU (°C)' :  'Temp_Heat_Exch_CCU','Temp. Nacelle (°C)' :  'Temp_Nacelle','Temp. Trafo 1 690 V (°C)' :  'Temp_Trafo_1','Temp. Trafo 2 690 V (°C)' :  'Temp_Trafo_2','Temp. Trafo 3 690 V (°C)' :  'Temp_Trafo_3','Temp. battery box 1 (°C)' :  'Temp_battery_box_1','Temp. battery box 2 (°C)' :  'Temp_battery_box_2','Temp. battery box 3 (°C)' :  'Temp_battery_box_3','Temp. bearing A (°C)' :  'Temp_bearing_A','Temp. bearing B (°C)' :  'Temp_bearing_B','Temp. generator 1 (°C)' :  'Temp_generator_1','Temp. generator 2 (°C)' :  'Temp_generator_2','Temp. generator cooling air (°C)' :  'Temp_generator_cool_air','Temperatue 2nd Outside (°C)' :  'Temp_2nd_Outside','Temperature MCC cabinet (°C)' :  'Temp_MCC_cabinet','Temperature slip ring (°C)' :  'Temp_slip_ring','Temperature top box (°C)' :  'Temp_top_box','TorqueOffset tower feedback (Nm)' :  'Torque_Offset_tower_feedback_Nm','Tower acc. lateral raw (mm/s²)' :  'Tower_acc_lateral_raw_mm_s2','Tower acc. normal raw (mm/s²)' :  'Tower_acc_normal_raw_mm_s2','Tower deflection (ms)' :  'Tower_deflection_ms','Turbine State' :  'Turbine_State','Voltage A-N (V)' :  'Voltage_A_N','Voltage B-N (V)' :  'Voltage_B_N','Voltage C-N (V)' :  'Voltage_C_N','Wind deviation 1 sec. (°)' :  'Wind_dev_1_sec','Wind deviation 10 sec. (°)' :  'Wind_dev_10_sec','Wind deviation 2, 1 sec. (°)' :  'Wind_dev_2_1_sec','Wind deviation 2, 10 sec. (°)' :  'Wind_dev_2_10_sec','Wind speed (m/s)' :  'Wind_speed_m_s','Windspeed a. 2 (m/s)' :  'Windspeed_a2_m_s','n-set 1 (rpm)' :  'nset_1_rpm','n-set 1 (rpm).1' :  'nset_1_rpm_1','n-set 2 (rpm)' :  'nset_2_rpm','n-set 2 (rpm).1' :  'nset_2_rpm_1'
              ,'10s wind speed average (m/s)' : '10s_wind_speed_avg_m_s','10s power average (kW)':'10s_power_avg',
'Der. Thrust with respect to pitch (kN/rad)':'Thrust_respect_pitch_kN_rad',
'Delta Pitch ATC (°)':'Delta_Pitch_ATC','Filtered Wind speed estimator (m/s)':'Filtered_Wind_speed_est_m_s',
'External power limit (kW)':'Ext_power_lmt_kW',
'Estimated Thrust (kN)':'Est_Thrust_kN',
'Der. Thrust with respet to wind speed (kNs/m)':'Thrust_respect_windspeed_kNs_m',
'Der. Thrust with respect to rotor speed (kNs/rad)':'Thrust_respect_rotorspeed_kNs_rad',
'Internal power limit (kW)':'Interal_power_lmt',
'Maximal power increase (kW/s)':'Max_power_inc',
'Maximal power decrease (kW/s)':'Max_power_dec',
'Minimum Pitch ATC (°)':'Min_Pitch_ATC',
'Possible power sent to WFMS (kW)':'Poss_power_WFMS_kW',
'Rotor acceleration estimator (rad/s²)':'Rotor_acc_est_rad_s2',
'Thrust limit ATC (kN)':'Thrust_limit_ATC_kN',
'Thrust standard dev. ATC (kN)':'Thrust_std_dev_ATC_kN',
'Thrust level ATC (kN)':'Thrust_std_level_ATC_kN',
'Thrust extreme ATC (kN)':'Thrust_ext_ATC_kN',
'Thrust extrapolated ATC (kN)':'Thrust_extrapolated_ATC_kN',
'Wind acceleration estimator (m/s²)':'Wind_acc_est_m_s2',
'Wind speed estimator (m/s)':'Wind_speed_est_m_s2',
'Event ID':'Event_ID',
'Fault':'Events_binary',                
'Lost Production (kWh)':'Lost_Production_kWh' }
#combined_data.head()
logs = combined_data.rename(columns=renamecolumns)
lab_col=['Period','Turbine_Number','Events_binary','Event_ID','Category','Description','Lost_Production_kWh']
Labels_df=logs[lab_col]
Labels_df.columns=['Period','Turbine_Number','Events_binary','Event_ID','Category','Description','Lost_Production_kWh']
Features_df=logs.drop(['Turbine_Number','Event_ID','Category','Description','Lost_Production_kWh'],axis=1)
for i in logs.Turbine_Number.unique():
    print(i)
    df_turbine=logs[logs.Turbine_Number==i]
    lab_turbine=Labels_df[Labels_df.Turbine_Number==i]
    df_turbine=df_turbine.drop('Turbine_Number',axis=1)
    Upload_Download_Pickle().save_dataset_pickle(prepath,  'Features'+str(i), df_turbine)
    Upload_Download_Pickle().save_dataset_pickle(prepath,  'Labels'+str(i), lab_turbine)
        

In [ ]:
#Combine Multiple Identical Datasets
Combine_Datasets().multi_combine_dataset()

In [ ]:
#Sonrasında Imputation uygula bir verilen değerler için bir de np.nan değerler için:
Basic_Imputation().impute_before_split()

Features_df_new=Upload_Download_Pickle().download_pickle(cpath, 'Features_df_new')
Features_List_new=Upload_Download_Pickle().download_pickle(cpath, 'Features_List_new')
#Fillna values with -999999
Features_df_new=Features_df_new.fillna(-999999)
Features_list_new_v1=[]
for data in Features_List_new:    
    data=data.fillna(-999999)
    Features_list_new_v1.append(data)
    
Upload_Download_Pickle().save_dataset_pickle(cpath,'Features_df_new',Features_df_new)
Upload_Download_Pickle().save_dataset_pickle(cpath,'Features_List_new',Features_list_new_v1)

#v2 oluşturuyoruz ama henüz kullanmıyoruz.
Default_values=[-999999,-888888]
Features_df_new_v2=Features_df_new.replace(Default_values, np.nan)
Features_list_new_v2=[]
for data in Features_list_new_v1:    
    data=data.replace(Default_values, np.nan)
    Features_list_new_v2.append(data)
#v2 nan olarak kaydediyoruz:  
Upload_Download_Pickle().save_dataset_pickle(cpath,'Features_df_nan',Features_df_new_v2)
Upload_Download_Pickle().save_dataset_pickle(cpath,'Features_List_nan',Features_list_new_v2)
#Split Train Test (Horizontal ayır)
Split_Train_Test().split_train_testset()
#Burda anomaly ve nan değerleri için exstra imputation yapılabilir.
#tum default değerleri nan ile değiştir. meta data muhabbetine bak!
X_test=Upload_Download_Pickle().download_pickle(path, 'X_test')
y_test=Upload_Download_Pickle().download_pickle(path, 'y_test')
X_trains=Upload_Download_Pickle().download_pickle(path, 'X_trains')
y_trains=Upload_Download_Pickle().download_pickle(path, 'y_trains')
#default değerler nan ile değiştirme kodu?

In [ ]:
from datetime import datetime, timedelta
LogsT02['onedayPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=1)
LogsT02['fivedaysPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=5)
LogsT02['fiftheendaysPeriod']=LogsT02['Period'].astype('datetime64[ns]')- timedelta(days=15)
LogsT02=LogsT02.set_index('Period')
tag=LogsT02[LogsT02['Events_binary']==1]

for i in tag.index:
    print(i)
    print(tag.loc[i,'onedayPeriod'])
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'fiftheendaysPeriod']),'EVENT_fifthteen',0)    
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'fivedaysPeriod']),'EVENT_five',LogsT02['Events_binary'])  
    LogsT02['New_Pitch_systemEv']=np.where((LogsT02.index<=i)&(LogsT02.index>tag.loc[i,'onedayPeriod']),'EVENT_one',LogsT02['Events_binary'])
  
    

In [ ]:
Normalize().featureExtraction()

In [ ]:
Rolling().featureExtraction()

In [10]:
#Oluşturulan featureları birleştir yaz.
X_trains_extNorm1=Upload_Download_Pickle().download_pickle(path, 'X_trains_extNorm1')
X_trains_extRoll1=Upload_Download_Pickle().download_pickle(path, 'X_trains_extRoll1')
X_trains_extRoll2=Upload_Download_Pickle().download_pickle(path, 'X_trains_extRoll2')
train=pd.concat([X_trains_extNorm1,X_trains_extRoll1])
train1=pd.concat([train,X_trains_extRoll2])
Upload_Download_Pickle().save_dataset_pickle(cpath,'X_trains_ext',train1)
del X_trains_extNorm1,X_trains_extRoll1,X_trains_extRoll2
#Oluşturulan featureları birleştir yaz.
X_test_extNorm1=Upload_Download_Pickle().download_pickle(path, 'X_test_extNorm1')
X_test_extRoll1=Upload_Download_Pickle().download_pickle(path, 'X_test_extRoll1')
X_test_extRoll2=Upload_Download_Pickle().download_pickle(path, 'X_test_extRoll2')
test=pd.concat([X_test_extNorm1,X_test_extRoll1])
test1=pd.concat([test,X_test_extRoll2])
#yeni pickle'a yaz
    

Upload_Download_Pickle().save_dataset_pickle(cpath,'X_test_ext',test1)
del X_test_extNorm1,X_test_extRoll1,X_test_extRoll2


#List için Oluşturulan featureları birleştir yaz.
#Oluşturulan featureları birleştir yaz.
X_trains_list_extNorm1=Upload_Download_Pickle().download_pickle(path, 'X_trains_list_extNorm1')
X_trains_list_extRoll1=Upload_Download_Pickle().download_pickle(path, 'X_trains_list_extRoll1')
X_trains_list_extRoll2=Upload_Download_Pickle().download_pickle(path, 'X_trains_list_extRoll2')
X_trains_list_ext=[]
for idx,data in enumerate(X_trains_list_extNorm1):
    print(idx)
    trains_list=pd.concat([data,X_trains_list_extRoll1[idx]])
    trains_list_1=pd.concat([trains_list,X_trains_list_extRoll2[idx]])
    X_trains_list_ext.append(trains_list_1)    
Upload_Download_Pickle().save_dataset_pickle(cpath,'X_trains_list_ext',X_trains_list_ext)
del X_trains_list_ext,X_trains_list_extNorm1,X_trains_list_extRoll1,X_trains_list_extRoll2

#List için Oluşturulan featureları birleştir yaz.
#Oluşturulan featureları birleştir yaz.
X_test_list_extNorm1=Upload_Download_Pickle().download_pickle(path, 'X_test_list_extNorm1')
X_test_list_extRoll1=Upload_Download_Pickle().download_pickle(path, 'X_test_list_extRoll1')
X_test_list_extRoll2=Upload_Download_Pickle().download_pickle(path, 'X_test_list_extRoll2')
X_test_list_ext=[]
for idx,data in enumerate(X_test_list_extNorm1):
    print(idx)
    test_list=pd.concat([data,X_test_list_extRoll1[idx]])
    test_list_1=pd.concat([test_list,X_test_list_extRoll2[idx]])
    X_test_list_ext.append(test_list_1)    
Upload_Download_Pickle().save_dataset_pickle(cpath,'X_test_list_ext',X_test_list_ext)
del X_test_list_ext,X_test_list_extNorm1,X_test_list_extRoll1,X_test_list_extRoll2

D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To ret

In [8]:
Basic_Elemination().featureSelection() 
#Feature Extraction: Normalize Smooth diff etc.
#Sonrasında Feature Selection bunu yaparken nan değerleri default değerlerle dolu halini kullan.
#Fault Detection Çalıştır. bunu yaparken de mümkünse proba kullan prob sonuçlarının yerine!! zor iş brace yourself 
# 1 gün öncesine kadar tagle öyle bakalım.

metadata already created.
True
Create new metadata and save:
self.metaconf:  meta
                                             VARIABLE     TYPE STATUS
0                             Blade2_act_val_A_degree  NUMERIC   KEEP
1                             Blade3_act_val_A_degree  NUMERIC   KEEP
2                               Blade1_set_val_degree  NUMERIC   KEEP
3                               Blade2_set_val_degree  NUMERIC   KEEP
4                             Blade1_act_val_A_degree  NUMERIC   KEEP
5                             Blade1_act_val_B_degree  NUMERIC   KEEP
6                             Blade2_act_val_B_degree  NUMERIC   KEEP
7                             Blade3_act_val_B_degree  NUMERIC   KEEP
8                                  Rotor_angle_degree  NUMERIC   KEEP
9                              10s_wind_speed_avg_m_s  NUMERIC   KEEP
10                                      10s_power_avg  NUMERIC   KEEP
11                                        Current_A_A  NUMERIC   KEEP
12      

not_zero:                          count  number_of_defaults
Scope_CH_1_norm        632448              632448
Scope_CH_2_norm        632448              632448
Scope_CH_3_norm        632448              632448
Temp_MCC_cabinet_norm  632448              632448
Temp_2nd_Outside_norm  632448              632448


..\src\featureProcessing\Feature_Selection\Basic_Elemination.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_zero['default_percentage'] = not_zero['number_of_defaults']/not_zero['count']


1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
meta_same:                                              VARIABLE     TYPE   STATUS
0                             Blade2_act_val_A_degree  NUMERIC     KEEP
1                             Blade3_act_val_A_degree  NUMERIC     KEEP
2                               Blade1_set_val_degree  NUMERIC     KEEP
3                               Blade2_set_val_degree  NUMERIC     KEEP
4                             Blade1_act_val_A_degree  NUMERIC     KEEP
5                             Blade1_act_val_B_degree  NUMERIC     KEEP
6                             Blade2_act_val_B_degree  NUMERIC     KEEP
7                             Blade3_act_val_B_degree  NUMERIC     KEEP
8                                  Rotor_angle_degree  NUMERIC     KEEP
9                              10s_wind_speed_avg_m_s  NUMERIC     KEEP
10                                      10s_power_avg  NUMERIC     KEEP
11                                        Current_A_A  NUMERIC     KE

Blade1_set_val_degree
Blade2_set_val_degree
Blade1_act_val_A_degree
Blade1_act_val_B_degree
Blade2_act_val_B_degree
Blade3_act_val_B_degree
Rotor_angle_degree
10s_wind_speed_avg_m_s
10s_power_avg
Current_A_A
Current_B_A
Current_C_A
Blade3_set_val_degree
CCU_Circuit_breaker_cutins
CCU_power_kW
CCU_reactive_power_kVAr
CCU_voltage_V
CCU_current_A
BuddyYawOffset_degree
Thrust_respect_pitch_kN_rad
Delta_Pitch_ATC
CCU_Generator_speed_rpm
Generator_rpm
Drive_train_acc_mm_s2
Hydraulic_prepressure_bar
Generator_temp_3_C
Filtered_Wind_speed_est_m_s
Ext_power_lmt_kW
Drip_loop_power_cable_temp_1
Drip_loop_power_cable_temp_2
Est_Thrust_kN
Thrust_respect_windspeed_kNs_m
Thrust_respect_rotorspeed_kNs_rad
Nacelle_Position
Line_freq_Hz
Mom_D_direction
Mainframe_acc_lateral
Mom_D_filtered
Mom_Q_filtered
Mainframe_acc_vertical
Mom_Q_direction
Interal_power_lmt
Max_power_inc
Max_power_dec
Min_Pitch_ATC
Power_kW
Nacelle_rev
nset_1_rpm
nset_2_rpm
Operating_state
Particle_counter
Pitch_offset_1_ALC_deg
Pitch

Shaft_bearing1_temp_C_roll_mean
Shaft_bearing2_temp_C_roll_mean
Tower_base_temp_C_roll_mean
Temp_Trafo_2_roll_mean
Temp_Heat_Exch_CCU_roll_mean
Temp_Trafo_1_roll_mean
Temp_Trafo_3_roll_mean
Temp_MCC_cabinet_roll_mean
Temp_2nd_Outside_roll_mean
Torque_act_Percent_roll_mean
Torque_set_Percent_roll_mean
Torque_Offset_tower_feedback_Nm_roll_mean
Tower_acc_normal_raw_mm_s2_roll_mean
Tower_acc_lateral_raw_mm_s2_roll_mean
Temp_top_box_roll_mean
Temp_slip_ring_roll_mean
Thrust_limit_ATC_kN_roll_mean
Thrust_std_dev_ATC_kN_roll_mean
Thrust_std_level_ATC_kN_roll_mean
Thrust_ext_ATC_kN_roll_mean
Thrust_extrapolated_ATC_kN_roll_mean
Tower_deflection_ms_roll_mean
Voltage_A_N_roll_mean
Voltage_B_N_roll_mean
Voltage_C_N_roll_mean
Wind_dev_1_sec_roll_mean
Wind_dev_10_sec_roll_mean
Tower_acc_normal_roll_mean
Tower_acc_lateral_roll_mean
Wind_dev_2_1_sec_roll_mean
Wind_dev_2_10_sec_roll_mean
Turbine_State_roll_mean
Wind_acc_est_m_s2_roll_mean
Wind_speed_m_s_roll_mean
Windspeed_a1_m_s_roll_mean
Windspeed_a

X_trains.shape: (632448, 552)
X_train_new.shape: (632448, 476)
(632448, 292)
292
final_meta                                              VARIABLE     TYPE  \
0                             Blade2_act_val_A_degree  NUMERIC   
1                             Blade3_act_val_A_degree  NUMERIC   
2                               Blade1_set_val_degree  NUMERIC   
3                               Blade2_set_val_degree  NUMERIC   
4                             Blade1_act_val_A_degree  NUMERIC   
5                             Blade1_act_val_B_degree  NUMERIC   
6                             Blade2_act_val_B_degree  NUMERIC   
7                             Blade3_act_val_B_degree  NUMERIC   
8                                  Rotor_angle_degree  NUMERIC   
9                              10s_wind_speed_avg_m_s  NUMERIC   
10                                      10s_power_avg  NUMERIC   
11                                        Current_A_A  NUMERIC   
12                                        Current_

In [9]:
meta=Upload_Download_Pickle().download_pickle(path, 'meta')

In [10]:
meta.groupby('STATUS').count()

,VARIABLE,TYPE
STATUS,,
DROP_all_same,46,46
DROP_many_unique,16,16
DROP_na,14,14
DROP_variance_smaller,184,184
KEEP,292,292


In [11]:
meta[meta['STATUS']!='KEEP']

,VARIABLE,TYPE,STATUS
2,Blade1_set_val_degree,NUMERIC,DROP_variance_smaller
3,Blade2_set_val_degree,NUMERIC,DROP_variance_smaller
11,Current_A_A,NUMERIC,DROP_variance_smaller
12,Current_B_A,NUMERIC,DROP_variance_smaller
13,Current_C_A,NUMERIC,DROP_variance_smaller
14,Blade3_set_val_degree,NUMERIC,DROP_variance_smaller
15,CCU_Circuit_breaker_cutins,NUMERIC,DROP_variance_smaller
20,BuddyYawOffset_degree,NUMERIC,DROP_all_same
22,Delta_Pitch_ATC,NUMERIC,DROP_all_same
24,Generator_rpm,NUMERIC,DROP_variance_smaller


In [12]:
#EDA için 
# Power ve wind fault/normal her turbine için çizdir.Pitch angle
#1.farklı stateler ile (Turbine State, Operating state, State and Faults ) Voltage ve Power değerleri için dağılım ve aralıklara bakılacak


In [13]:
#Feature Selection with Hybrid method # char değişkenleri dummification
Hybrid().featureSelection()

self.metaconf:  meta
metadata already created.
False
self.meta                                              VARIABLE     TYPE  \
0                             Blade2_act_val_A_degree  NUMERIC   
1                             Blade3_act_val_A_degree  NUMERIC   
2                               Blade1_set_val_degree  NUMERIC   
3                               Blade2_set_val_degree  NUMERIC   
4                             Blade1_act_val_A_degree  NUMERIC   
5                             Blade1_act_val_B_degree  NUMERIC   
6                             Blade2_act_val_B_degree  NUMERIC   
7                             Blade3_act_val_B_degree  NUMERIC   
8                                  Rotor_angle_degree  NUMERIC   
9                              10s_wind_speed_avg_m_s  NUMERIC   
10                                      10s_power_avg  NUMERIC   
11                                        Current_A_A  NUMERIC   
12                                        Current_B_A  NUMERIC   
13           

D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in

D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in

D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in

D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Users\sezgis\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in

...

meta                                              VARIABLE     TYPE  \
0                             Blade2_act_val_A_degree  NUMERIC   
1                             Blade3_act_val_A_degree  NUMERIC   
2                               Blade1_set_val_degree  NUMERIC   
3                               Blade2_set_val_degree  NUMERIC   
4                             Blade1_act_val_A_degree  NUMERIC   
5                             Blade1_act_val_B_degree  NUMERIC   
6                             Blade2_act_val_B_degree  NUMERIC   
7                             Blade3_act_val_B_degree  NUMERIC   
8                                  Rotor_angle_degree  NUMERIC   
9                              10s_wind_speed_avg_m_s  NUMERIC   
10                                      10s_power_avg  NUMERIC   
11                                        Current_A_A  NUMERIC   
12                                        Current_B_A  NUMERIC   
13                                        Current_C_A  NUMERIC   
14   

                                             VARIABLE     TYPE STATUS
0                             Blade2_act_val_A_degree  NUMERIC   KEEP
1                             Blade3_act_val_A_degree  NUMERIC   KEEP
4                             Blade1_act_val_A_degree  NUMERIC   KEEP
5                             Blade1_act_val_B_degree  NUMERIC   KEEP
6                             Blade2_act_val_B_degree  NUMERIC   KEEP
7                             Blade3_act_val_B_degree  NUMERIC   KEEP
8                                  Rotor_angle_degree  NUMERIC   KEEP
9                              10s_wind_speed_avg_m_s  NUMERIC   KEEP
10                                      10s_power_avg  NUMERIC   KEEP
16                                       CCU_power_kW  NUMERIC   KEEP
17                            CCU_reactive_power_kVAr  NUMERIC   KEEP
18                                      CCU_voltage_V  NUMERIC   KEEP
19                                      CCU_current_A  NUMERIC   KEEP
21                  

temp_features_new 10s_power_avg_norm_roll_mean
temp_features_new ['10s_power_avg_norm_roll_mean']
basic                                              VARIABLE     TYPE STATUS
0                             Blade2_act_val_A_degree  NUMERIC   KEEP
1                             Blade3_act_val_A_degree  NUMERIC   KEEP
4                             Blade1_act_val_A_degree  NUMERIC   KEEP
5                             Blade1_act_val_B_degree  NUMERIC   KEEP
6                             Blade2_act_val_B_degree  NUMERIC   KEEP
7                             Blade3_act_val_B_degree  NUMERIC   KEEP
8                                  Rotor_angle_degree  NUMERIC   KEEP
9                              10s_wind_speed_avg_m_s  NUMERIC   KEEP
10                                      10s_power_avg  NUMERIC   KEEP
16                                       CCU_power_kW  NUMERIC   KEEP
17                            CCU_reactive_power_kVAr  NUMERIC   KEEP
18                                      CCU_voltage_V  N

546                Wind_speed_est_m_s2_norm_roll_mean  NUMERIC   KEEP
                                             VARIABLE     TYPE STATUS
0                             Blade2_act_val_A_degree  NUMERIC   KEEP
1                             Blade3_act_val_A_degree  NUMERIC   KEEP
4                             Blade1_act_val_A_degree  NUMERIC   KEEP
5                             Blade1_act_val_B_degree  NUMERIC   KEEP
6                             Blade2_act_val_B_degree  NUMERIC   KEEP
7                             Blade3_act_val_B_degree  NUMERIC   KEEP
8                                  Rotor_angle_degree  NUMERIC   KEEP
9                              10s_wind_speed_avg_m_s  NUMERIC   KEEP
10                                      10s_power_avg  NUMERIC   KEEP
16                                       CCU_power_kW  NUMERIC   KEEP
17                            CCU_reactive_power_kVAr  NUMERIC   KEEP
18                                      CCU_voltage_V  NUMERIC   KEEP
19                  

final_meta                                              VARIABLE     TYPE  \
0                             Blade2_act_val_A_degree  NUMERIC   
1                             Blade3_act_val_A_degree  NUMERIC   
2                               Blade1_set_val_degree  NUMERIC   
3                               Blade2_set_val_degree  NUMERIC   
4                             Blade1_act_val_A_degree  NUMERIC   
5                             Blade1_act_val_B_degree  NUMERIC   
6                             Blade2_act_val_B_degree  NUMERIC   
7                             Blade3_act_val_B_degree  NUMERIC   
8                                  Rotor_angle_degree  NUMERIC   
9                              10s_wind_speed_avg_m_s  NUMERIC   
10                                      10s_power_avg  NUMERIC   
11                                        Current_A_A  NUMERIC   
12                                        Current_B_A  NUMERIC   
13                                        Current_C_A  NUMERIC   

In [ ]:
Features_df[Features_df['Events_binary']==1]['State_Fault'].unique()

In [ ]:
#Split Hold-0ut Dataset
Split_Train_Test().split_train_testset()

In [ ]:
X_test=Upload_Download_Pickle().download_pickle(path, 'X_test')
y_test=Upload_Download_Pickle().download_pickle(path, 'y_test')
X_tests_list=Upload_Download_Pickle().download_pickle(path, 'X_tests_list')
y_trains=Upload_Download_Pickle().download_pickle(path, 'y_trains')

In [ ]:
Normalize().featureExtraction()

In [ ]:
Basic_Elemination().featureSelection()

In [ ]:
meta=Upload_Download_Pickle().download_pickle(path, 'meta')

In [ ]:
meta.groupby('STATUS').count()

In [ ]:
#Feature Selection with Hybrid method
Hybrid().featureSelection()

In [ ]:
meta=Upload_Download_Pickle().download_pickle(path, 'meta')

In [ ]:
meta.groupby('STATUS').count()

In [1]:
X_trains_ext=Upload_Download_Pickle().download_pickle(path, 'X_trains_ext')
X_trains_list_ext=Upload_Download_Pickle().download_pickle(path, 'X_trains_list_ext')
X_test_ext=Upload_Download_Pickle().download_pickle(path, 'X_test_ext')
X_test_list_ext=Upload_Download_Pickle().download_pickle(path, 'X_test_list_ext')
y_tests_list=Upload_Download_Pickle().download_pickle(path, 'y_tests_list')

NameError: name 'Upload_Download_Pickle' is not defined

In [ ]:
X_trains_list[0].head()

In [ ]:
len(y_tests_list)

In [ ]:
X_trains_ext.dtypes

In [16]:
XGBoost_Tune().Model()

{'max_depth': [5, 7, 8], 'min_child_weight': [0.01, 1.0]}


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "str"

In [ ]:
# Create & Predict 
XGBoost_Tune().Model()
LightGBM_Tune().Model()
CatBoost_Tune().Model()
#Random_Forest_Tune().Model()
#Voting().Model()
#SVC_Tune().Model()

In [ ]:
Random_Forest_Tune().Model()

In [17]:
processed_path=Config_Paths().get_processed_path()
selectedfeatures = Upload_Download_Pickle().download_pickle(processed_path, 'selectedfeatures')


In [18]:
selectedfeatures

Index(['State_Fault_norm_roll_mean', 'Turbine_State_norm_roll_mean',
       'Blade1_act_val_B_degree', 'Blade2_act_val_A_degree_norm_roll_mean',
       'Blade3_act_val_B_degree_norm', 'Scope_CH_4_norm_roll_mean',
       'Particle_counter_norm', 'Wind_speed_est_m_s2_norm',
       'Power_kW_norm_roll_mean', 'Prox_sensor45_mm',
       '10s_power_avg_norm_roll_mean', 'Pitch_offset_3_ALC_deg_norm_roll_mean',
       'Nacelle_rev_roll_mean', 'Thrust_limit_ATC_kN',
       'Filtered_Wind_speed_est_m_s_norm', 'Filtered_Wind_speed_est_m_s',
       'CCU_voltage_V_norm', 'Temp_battery_box_2_roll_mean', 'Max_power_inc',
       'Rotor_lock_position_roll_mean'],
      dtype='object')

In [ ]:
y_pred_proba = Upload_Download_Pickle().download_pickle(modelpath, 'XGboost_classification_proba_each_prediction.pckl')

In [ ]:
for i in y_pred_proba:
    xx=pd.DataFrame(i)
    print('xx:',xx[1])

In [ ]:
# Create Model Metrics according to test sets
XGBoost_Tune().Model_Metrics('classification')

In [ ]:
# Create Model Metrics according to test sets
XGBoost_Tune().Model_Metrics('classification')
LightGBM_Tune().Model_Metrics('classification')
CatBoost_Tune().Model_Metrics('classification')
Random_Forest_Tune().Model_Metrics('classification')
#SVC_Tune().Model_Metrics('classification')

In [ ]:
# Plot Results of Grid Search
XGBoost_Tune().grid_plot()
LightGBM_Tune().grid_plot()
CatBoost_Tune().grid_plot()
Random_Forest_Tune().grid_plot()
SVC_Tune().grid_plot()

In [ ]:
# Plot Results of Grid Search
XGBoost_Tune().grid_plot()